<font color='blue' size=5><b>Комментарий ревьюера</b></font>

<font color='blue'>Привет, Артем! Меня зовут Павел Григорьев, и я буду проверять этот проект.<br>Моя основная цель - не указать на совершённые тобой ошибки, а поделиться своим опытом и помочь тебе совершенствоваться как профессионалу.<br>Спасибо за проделанную работу! Предлагаю общаться на «ты».</font>
<details>
	<summary><u>Инструкция по организационным моментам (кликабельно)</u></summary>
<font color='blue'>Я буду использовать различные цвета, чтобы было удобнее воспринимать мои комментарии:</font>


---


<font color='blue'>синий текст - просто текст комментария</font>

<font color='green'>✔️ и зеленый текст - все отлично</font>

<font color='orange'>⚠️ и оранжевый текст - сделано все правильно, однако есть рекомендации, на что стоит обратить внимание</font>

<font color='red'>❌ и красный текст - есть недочеты</font>


</details>    
    </br>
<font color='blue'>Пожалуйста, не удаляй мои комментарии в случае возврата работы, так будет проще разобраться, какие были недочеты, а также сразу увидеть исправленное. </font>

Ответы на мои комментарии лучше тоже помечать.
Например: <font color='purple'><b>Комментарий студента</b></font>

<font color='blue'><b>Давай смотреть, что получилось!</b></font>

<font color='blue' size=3><b>Комментарий ревьюера 2</b></font>

<font color='blue'>Привет еще раз. Спасибо, за исправления. Оформление комментариев по работе сохраняется. Только обозначим, что это вторая итерация.</font> 

# Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

In [1]:
# Обновим pyspark до версии 3.4.0 для доступа к функции median
!pip install --upgrade pyspark>=3.4.0

import pyspark
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import isnan, when, count, col, median, rand

RANDOM_SEED = 3074

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Хорошее оформление импортов! \
Импорты собраны в одной ячейке, разделены на функциональные группы пустой строкой.</font>

## Подготовка данных

### Инициализируем локальную Spark-сессию

In [2]:
spark = SparkSession.builder.master("local").appName("Median house value Prediction").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/03 19:41:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Прочитаем содержимое файла `/datasets/housing.csv`

In [3]:
schema = StructType([
    StructField('longitude', DoubleType(), nullable=True),
    StructField('latitude', DoubleType(), nullable=True),
    StructField('housing_median_age', DoubleType(), nullable=True),
    StructField('total_rooms', DoubleType(), nullable=True),
    StructField('total_bedrooms', DoubleType(), nullable=True),
    StructField('population', DoubleType(), nullable=True),
    StructField('households', DoubleType(), nullable=True),
    StructField('median_income', DoubleType(), nullable=True),
    StructField('median_house_value', DoubleType(), nullable=True),
    StructField('ocean_proximity', StringType(), nullable=True)]
)

data = spark.read.load('/datasets/housing.csv', format="csv", sep=',', inferSchema=False, schema=schema, header='true')

### Выведем тип данных колонок и несколько наугад выбранных строк датасета

In [4]:
data.printSchema()
data.orderBy(rand()).limit(5).show(vertical=True)

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



-RECORD 0-----------------------
 longitude          | -121.2    
 latitude           | 37.79     
 housing_median_age | 36.0      
 total_rooms        | 866.0     
 total_bedrooms     | 160.0     
 population         | 502.0     
 households         | 149.0     
 median_income      | 2.4798    
 median_house_value | 101500.0  
 ocean_proximity    | INLAND    
-RECORD 1-----------------------
 longitude          | -120.86   
 latitude           | 37.73     
 housing_median_age | 27.0      
 total_rooms        | 508.0     
 total_bedrooms     | 93.0      
 population         | 263.0     
 households         | 81.0      
 median_income      | 3.1136    
 median_house_value | 183300.0  
 ocean_proximity    | INLAND    
-RECORD 2-----------------------
 longitude          | -118.49   
 latitude           | 34.26     
 housing_median_age | 27.0      
 total_rooms        | 2722.0    
 total_bedrooms     | 468.0     
 population         | 1164.0    
 households         | 419.0     
 median_in

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Отлично!  Выведены типы данных и первые строчки таблицы.</font>

### Исследуем данные на наличие пропусков и заполним их, выбрав значения по своему усмотрению

In [5]:
#check
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show(vertical=True)

-RECORD 0-----------------
 longitude          | 0   
 latitude           | 0   
 housing_median_age | 0   
 total_rooms        | 0   
 total_bedrooms     | 207 
 population         | 0   
 households         | 0   
 median_income      | 0   
 median_house_value | 0   
 ocean_proximity    | 0   



<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'>Обнаружены пропуски!</font>

In [6]:
#check
summary = data.select('total_bedrooms').summary()
summary.show()

+-------+------------------+
|summary|    total_bedrooms|
+-------+------------------+
|  count|             20433|
|   mean| 537.8705525375618|
| stddev|421.38507007403115|
|    min|               1.0|
|    25%|             296.0|
|    50%|             435.0|
|    75%|             647.0|
|    max|            6445.0|
+-------+------------------+



Можно заметить, что в столбце `total_bedrooms` присутствуют аномальные значения. В таком случае разумнее будет заполнить пропуски медианным значением вместо среднего

In [7]:
data = data.na.fill(value=float(summary.where(col('summary') == '50%').collect()[0].total_bedrooms), subset=['total_bedrooms'])

In [8]:
#check
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show(vertical=True)

-RECORD 0-----------------
 longitude          | 0   
 latitude           | 0   
 housing_median_age | 0   
 total_rooms        | 0   
 total_bedrooms     | 0   
 population         | 0   
 households         | 0   
 median_income      | 0   
 median_house_value | 0   
 ocean_proximity    | 0   



<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'>Заполнили пропуски!</font>

### Соберём все преобразования признаков в один массив для дальнейшего использования в pipeline

In [9]:
indexer = StringIndexer(inputCols=['ocean_proximity'], outputCols=['ocean_proximity_idx']) 
encoder = OneHotEncoder(inputCols=['ocean_proximity_idx'], outputCols=['ocean_proximity_ohe'])
categorical_assembler = VectorAssembler(inputCols=['ocean_proximity_ohe'], outputCol='categorical_features')
numerical_assembler = VectorAssembler(inputCols=['longitude', 'latitude', 'housing_median_age', 'total_rooms',
                                                 'total_bedrooms', 'population', 'households', 'median_income'],
                                      outputCol='numerical_features')
scaler = StandardScaler(inputCol='numerical_features', outputCol='numerical_features_scaled')
features_assembler = VectorAssembler(inputCols=['categorical_features','numerical_features_scaled'], 
                                     outputCol='features')

transformations = [indexer, encoder, categorical_assembler, numerical_assembler, scaler, features_assembler]

<font color='blue'><b>Комментарий ревьюера 2: </b></font> ⚠️<a id='handleInvalid'></a>\
<font color='blue'>Бывает так, что в тестовых данных случается категория которой нет в обучающих. В StringIndexer есть три стратегии обработки незнакомых Лейблов.
1. Выбрасывать исключение (по умолчанию).
2. Игнорировать строки с незнакомыми лейблами.
3. Выделять для всех незнакомых лейблов отдельную категорию.

За это отвечает аргумент [handleInvalid](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.StringIndexer.html#pyspark.ml.feature.StringIndexer.handleInvalid), в который нужно передать "error","skip" или "keep" соответственно.</font>

<font color='blue'><b>Комментарий ревьюера: </b></font> ❌\
<font color='red'><s>Кодирование и Стандартизацию нужно проводить после разделения на выборки трансформерами обучеными на обучающих данных.</s></font>

<font color='purple'><b>Исправлено</b></font>

<font color='blue'><b>Комментарий ревьюера: </b></font> ⚠️\
<font color='blue'>Можно использовать [Pipeline](https://habr.com/ru/post/522296/)</font>

<font color='purple'><b>Исправлено. Добавлен Pipeline</b></font>

<font color='blue'><b>Комментарий ревьюера 2: </b></font> ✔️\
<font color='green'> Отлично, что используешь Pipeline, для предобработки. \
Утечки информации между выборками исключены, даже если применять кроссвалидацию!</font>

### Разделим данные на выборку для обучения и выборку для тестирования качества модели

In [10]:
train_data, test_data = data.randomSplit([.8,.2], seed=RANDOM_SEED)

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Здорово , что у нас есть выборка для тестов!</font>

## Обучение моделей

### Построим модель линейной регрессии, используя и числовые и категориальные переменные

In [11]:
lr = LinearRegression(featuresCol='features', labelCol='median_house_value')
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.elasticNetParam, [1, 0.1, 0.01]) \
    .addGrid(lr.maxIter, [10, 100, 1000]) \
    .addGrid(lr.solver, ['normal', 'l-bfgs']) \
    .build()
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(labelCol='median_house_value'),
                          numFolds=5,
                          parallelism=2)
                      
model = Pipeline(stages=transformations+[crossval]).fit(train_data)

23/11/03 19:41:36 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/11/03 19:43:21 WARN BlockManager: Asked to remove block broadcast_4318_piece0, which does not exist
23/11/03 19:44:23 WARN BlockManager: Asked to remove block broadcast_8136, which does not exist
23/11/03 19:55:30 WARN BlockManager: Asked to remove block broadcast_52883, which does not exist
23/11/03 19:56:34 WARN BlockManager: Asked to remove block broadcast_57509_piece0, which does not exist
23/11/03 20:11:46 WARN BlockManager: Asked to remove block broadcast_123119, which does not exist
23/11/03 20:20:51 WARN BlockManager: Asked to remove block broadcast_162841, which does not exist


<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Здорово , что используешь кроссвалидацию!</font>\
<font color='darkorange'> Но здесь точно нужен Pipeline для исключения утечек между обучающими и валидационными данными.</font>

<font color='purple'><b>Исправлено. Добавлен Pipeline</b></font>

<font color='blue'><b>Комментарий ревьюера 2: </b></font> ✔️\
<font color='green'> 👍</font>

### Построим модель линейной регрессии, используя только числовые переменные

In [12]:
lr = LinearRegression(featuresCol='numerical_features_scaled', labelCol='median_house_value')
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.elasticNetParam, [1, 0.1, 0.01]) \
    .addGrid(lr.maxIter, [10, 100, 1000]) \
    .addGrid(lr.solver, ['normal', 'l-bfgs']) \
    .build()
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(labelCol='median_house_value'),
                          numFolds=5,
                          parallelism=2)
                      
model_numerical = Pipeline(stages=transformations+[crossval]).fit(train_data)

## Анализ результатов

In [13]:
evaluator = RegressionEvaluator(labelCol='median_house_value')

### Выведем результаты работы модели линейной регрессии, обученной на полном наборе данных

In [14]:
print('model RMSE:', round(evaluator.setParams(metricName="rmse").evaluate(model.transform(test_data)), 4))
print('model MAE:', round(evaluator.setParams(metricName="mae").evaluate(model.transform(test_data)), 4))
print('model R2:', round(evaluator.setParams(metricName="r2").evaluate(model.transform(test_data)), 4))

model RMSE: 69661.9509
model MAE: 49966.071
model R2: 0.6314


### Выведем результаты работы модели линейной регрессии, обученной на неполном наборе данных (только количественные переменные)

In [15]:
print('model_numerical RMSE:', round(evaluator.setParams(metricName="rmse").evaluate(model_numerical.transform(test_data)), 4))
print('model_numerical MAE:', round(evaluator.setParams(metricName='mae').evaluate(model_numerical.transform(test_data)), 4))
print('model_numerical R2:', round(evaluator.setParams(metricName='r2').evaluate(model_numerical.transform(test_data)), 4))

model_numerical RMSE: 70894.1874
model_numerical MAE: 51307.8333
model_numerical R2: 0.6182


<font color='blue'><b>Комментарий ревьюера :</b></font> ✔️\
<font color='green'>Тестирование проведено верно.</font>

In [16]:
spark.stop()

<font color='blue'><b>Комментарий ревьюера: </b></font> ⚠️\
<font color='darkorange'>Не забывай останавливать сессию `spark.stop()`</font>

<font color='purple'><b>Исправлено</b></font>

<font color='blue'><b>Комментарий ревьюера 2: </b></font> ✔️\
<font color='green'>Здорово, что не забываешь останавливать сессию!</font>

### Вывод

Результаты работы модели линейной регрессии, обученной на полном наборе данных, чуть лучше, чем результаты работы модели, обученной на неполном наборе данных, по всем трём метрикам. Можно предположить, что существует зависимость между признаком `ocean_proximity` и целевым признаком `median_house_value` и первая модель успешно уловила эту взаимосвязь

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Очень приятно видеть вывод в конце проекта!\
Приведены ответы на главные вопросы проекта.</font>

<font color='blue'><b>Итоговый комментарий ревьюера</b></font>\
<font color='green'> Артем, хороший проект получился!
Большое спасибо за проделанную работу. Видно, что приложено много усилий.
</font>

<font color='blue'>Что нужно исправить:</font>
<ul><font color='red'>Стандартизация и Кодировка для train и test по отдельности обработчиком обученным на train.</font></ul>

<font color='blue'>Что можно сделать лучше:</font>
<font color='orange'>В работе я оставил несколько советов. Буду рад, если ты учтешь их.</font></ul>

<font color='blue'><b>Жду новую версию проекта :)</b></font>

<font color='blue'><b>Итоговый комментарий ревьюера 2</b></font>\
<font color='green'> Артем, проект принят! \
Все этапы пройдены. Все рекомендации учтены.\
Надеюсь, тебе понравился процесс выполнения и результат.</font> \
<font color='blue'><b>Спасибо, удачи в освоении профессии!</b></font>

<font color='green'>Немного полезного материала:
+  https://sparkbyexamples.com/pyspark-tutorial/, https://sparkbyexamples.com/
+  https://github.com/dvgodoy/handyspark
+  https://www.tutorialspoint.com/pyspark/index.htm
+  https://www.guru99.com/pyspark-tutorial.html
+  https://databricks.com/spark/getting-started-with-apache-spark/machine-learning#load-sample-data
</font>